In [2]:
# Reference: https://book.pythontips.com/en/latest/decorators.html
# Decorators are functions which modify the functionality of other functions.
# Two different decorators:
# -------function decorators (more common)
# -------class decorators

# Syntax of decorator
# So this is how the decorator syntax looks. A decorator is a function that takes another function as an argument
# and extends the behavior of that function without explicitly modifying it.
# In other words, it allows you to add new functionality to an existing function
@mydecorator
def dosomething():
    pass

NameError: name 'mydecorator' is not defined

In [3]:
# Let's first define a function
def print_name():
    print("Alex")

In [4]:
# Now we want to extend the behavior of this function. We can achieve this by using decorator
def start_end_decorator(func):
    
    def wrapper():
        print("Start")
        func()
        print("End")
    return wrapper

print_name = start_end_decorator(print_name)
print_name()

Start
Alex
End


In [5]:
# We can also do the same thing using annotation
@start_end_decorator
def print_name():
    print("Alex")
    
print_name()

Start
Alex
End


In [7]:
# Now let's try another application, which give us an error message because the decorator function is defined 
# with no arguments
@start_end_decorator
def add5(x):
    return x + 5

add5(10)

TypeError: wrapper() takes 0 positional arguments but 1 was given

In [9]:
# We can correct the above problem as below:
def start_end_decorator(func):
    
    def wrapper(*args, **kwargs):
        print("Start")
        func(*args, **kwargs)
        print("End")
    return wrapper

@start_end_decorator
def add5(x):
    return x + 10

add5(10)

Start
End


In [10]:
# Let's look at another problem
result = add5(10)
print(result)

Start
End
None


In [12]:
# Interestingly, it returns None instead of 15, that is because our wrapper function is defined w/o return
# We can fix this:
def start_end_decorator(func):
    
    def wrapper(*args, **kwargs):
        print("Start")
        result = func(*args, **kwargs)
        print("End")
        return result
    return wrapper

@start_end_decorator
def add5(x):
    return x + 5

result = add5(10)
print(result)

Start
End
15


In [15]:
# Now we have another problem, Python got confused because both help function and function name is 'wrapper'
print(help(add5))
print(add5.__name__) # the name should be 'add5', instead of wrapper

Help on function wrapper in module __main__:

wrapper(*args, **kwargs)

None
wrapper


In [17]:
# We can fix the problem above by using 'functools'
import functools

def start_end_decorator(func):
    
    # Here by adding '@functools.wraps(func)', this will now preserve the information of the used function
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        print("Start")
        result = func(*args, **kwargs)
        print("End")
        return result
    return wrapper

@start_end_decorator
def add5(x):
    return x + 5

print(help(add5))
print(add5.__name__)

Help on function add5 in module __main__:

add5(x)
    # Here by adding '@functools.wraps(func)', this will now preserve the information of the used function

None
add5


In [18]:
# The decorator in the above cell can be used as decorator template for all decorator functions:
import functools

def my_decorator(func):
    
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        # Do something before
        result = func(*args, **kwargs)
        # Do somehting afterwords
        return result
    return wrapper

In [19]:
# Let's try another example about decorators
def repeat(num_times):
    def decorator_repeat(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            for _ in range(num_times):
                result = func(*args, **kwargs)
            return result
        return wrapper
    return decorator_repeat

# Decorator with arguments
@repeat(num_times=3)
def greet(name):
    print(f'Hello {name}')
    
greet('Alex')

Hello Alex
Hello Alex
Hello Alex


In [24]:
# Nested decorators

def start_end_decorator(func):
    
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        print("Start")
        result = func(*args, **kwargs)
        print("End")
        return result
    return wrapper

# debug decorator
def debug(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        # Do something before
        args_repr = [repr(a) for a in args]
        kwargs_repr = [f"{k}={v!r}" for k, v in kwargs.items()]
        signature = ", ".join(args_repr + kwargs_repr)
        print(f"Calling {func.__name__}({signature})")
        # Original defined function
        result = func(*args, **kwargs)
        # Do something afterward
        print(f"{func.__name__!r} returned {result!r}")
        return result
    return wrapper

@debug
@start_end_decorator
def say_hello(name):
    greeting = f'Hello {name}'
    print(greeting)
    return greeting

say_hello('Alex')

Calling say_hello('Alex')
Start
Hello Alex
End
'say_hello' returned 'Hello Alex'


'Hello Alex'

In [35]:
# Class decorator
# We create a decorator that keeps track of the number of times a function being called

class CountCalls:
    
    def __init__(self, func):
        self.func = func
        self.num_calls = 0
        
    def __call__(self, *args, **kwargs):
        self.num_calls += 1
        print(f"This is executed {self.num_calls} times.")
        return self.func(*args, **kwargs)
        
@CountCalls
def say_hello():
    print("Hello")
    
say_hello()
say_hello()

This is executed 1 times.
Hello
This is executed 2 times.
Hello
